# Benchmark m4

How to fit several model and evaluate its predictions on three of subsets of M4 data. 


#### functools.partial

Return a new `partial` object which when called will behave like *func* called with the positional arguments args and keyword arguments keywords. 

- [Source](https://docs.python.org/2/library/functools.html)

## Methods  

Brief overview of the used methods

### `DeepAREstimator`

- [`gluonts.model.deepar`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

Construct a DeepAR estimator, implements an RNN-based model, close to the one described in

- Salinas, David, Valentin Flunkert, and Jan Gasthaus. “DeepAR: Probabilistic forecasting with autoregressive recurrent networks.” arXiv preprint arXiv:1704.04110 (2017).


### `MQCNNEstimator`

- [`gluonts.model.seq2seq`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.seq2seq.html)

An `MQDNNEstimator` with Convolutional Neural Network (CNN) as an encoder. Implements the MQ-CNN Forecaster, proposed in 

- Wen, Ruofeng, et al. “A multi-horizon quantile recurrent forecaster.” arXiv preprint arXiv:1711.11053 (2017).


### `SimpleFeedForwardEstimator`

- [`gluonts.model.simple_feedforward`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.simple_feedforward.html)

A simple multilayer perceptron model predicting the next target time-steps given the previous ones. 



In [6]:
!pip install gluonts

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# install gluonts (most recent version)
!pip install git+https://github.com/awslabs/gluon-ts.git

In [8]:
# import mxnet as mx
# from mxnet import gluon

# imports
import pandas as pd
import numpy as np

import pprint
from functools import partial

# gluonts imports
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.trainer import Trainer

# gluonts models
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator


OSError: libcudart.so.9.2: cannot open shared object file: No such file or directory

In [3]:
# gluonts version (important)
import gluonts
print(gluonts.__version__)

0.3.4.dev82+gd8f56f7


Number of series in m4 data: 

- Yearly: 23,000
- Quarterly: 24,000
- Monthly: 48,000
- Weekly: 359
- Daily: 4227
- Hourly: 414



In [4]:
epochs = 300
num_batches_per_epoch = 200

datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
    "m4_yearly",
]

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, 
            num_batches_per_epoch=num_batches_per_epoch,
            ctx=mx.Context("gpu")            
        ),
    ),
]

def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict

print("Data:\n", datasets)
print("Epochs:\n", epochs)
print("num_batches_per_epoch:\n", num_batches_per_epoch)

INFO:root:Using CPU


Data:
 ['m4_quarterly', 'm4_yearly']
Epochs:
 300
num_bathces_per_epoch:
 200


In [5]:
# estimators = [
#     partial(
#         SimpleFeedForwardEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
#     partial(
#         DeepAREstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
#     partial(
#         DeepAREstimator,
#         distr_output=PiecewiseLinearOutput(8),
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
# ]


In [6]:
if __name__ == "__main__":
    
    mx.random.seed(42)
    np.random.seed(42)
    
    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            print(estimator)
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())
    
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS","wQuantileLoss[0.5]", "wQuantileLoss[0.9]", ]]

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/200 [00:00<?, ?it/s]

functools.partial(<class 'gluonts.model.deepar._estimator.DeepAREstimator'>, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=300, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=200, patience=10, weight_decay=1e-08))
evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=300, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=200, patience=10, weight_decay=1e-08), use_feat_dynamic_real=Fal

INFO:root:Number of parameters in DeepARTrainingNetwork: 1230523
100%|██████████| 200/200 [00:05<00:00, 39.96it/s, avg_epoch_loss=7.68]
INFO:root:Epoch[0] Elapsed time 5.007 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.682601
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:05<00:00, 39.60it/s, avg_epoch_loss=7.19]
INFO:root:Epoch[1] Elapsed time 5.052 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.187988
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 40.28it/s, avg_epoch_loss=7.21]
INFO:root:Epoch[2] Elapsed time 4.967 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.209194
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:05<00:00, 39.57it/s, avg_epoch_loss=7.11]
INFO:root:Epoch[3] Elapsed time 5.057 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.109260
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 40.02it/s, avg_epoch_los

{'Coverage[0.1]': 0.10884895833333333,
 'Coverage[0.2]': 0.21906770833333333,
 'Coverage[0.3]': 0.3349947916666667,
 'Coverage[0.4]': 0.45615625,
 'Coverage[0.5]': 0.5709270833333333,
 'Coverage[0.6]': 0.6683802083333333,
 'Coverage[0.7]': 0.7649114583333333,
 'Coverage[0.8]': 0.8498541666666667,
 'Coverage[0.9]': 0.92378125,
 'MAE_Coverage': 0.044102430555555544,
 'MASE': 1.1704354483828787,
 'MSE': 1742971.5980532658,
 'MSIS': 11.3569489583177,
 'ND': 0.09311244459796786,
 'NRMSE': 0.2209809931098302,
 'QuantileLoss[0.1]': 48434974.1577034,
 'QuantileLoss[0.2]': 74674719.65602723,
 'QuantileLoss[0.3]': 91699179.06450501,
 'QuantileLoss[0.4]': 102145442.12934569,
 'QuantileLoss[0.5]': 106806870.84781647,
 'QuantileLoss[0.6]': 105547872.20766297,
 'QuantileLoss[0.7]': 98553543.65227357,
 'QuantileLoss[0.8]': 84212922.89341429,
 'QuantileLoss[0.9]': 58925975.760197446,
 'RMSE': 1320.2164966600235,
 'abs_error': 106806870.8767395,
 'abs_target_mean': 5974.344119287491,
 'abs_target_sum':

INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/200 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 1179723


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[23000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="12M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=6, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=300, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=200, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='12M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='23000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=6>, train=<gluonts.dataset.common.FileDataset object at 0x7f434

100%|██████████| 200/200 [00:04<00:00, 46.92it/s, avg_epoch_loss=7.51]
INFO:root:Epoch[0] Elapsed time 4.268 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.512975
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 46.77it/s, avg_epoch_loss=7.04]
INFO:root:Epoch[1] Elapsed time 4.278 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.038786
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 46.94it/s, avg_epoch_loss=7.16]
INFO:root:Epoch[2] Elapsed time 4.263 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.164348
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 46.92it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[3] Elapsed time 4.266 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.981029
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 200/200 [00:04<00:00, 47.06it/s, avg_epoch_loss=6.29]
INFO:root:Epoch[4] Elapsed time 4.251 seconds
INFO:root:E

{'Coverage[0.1]': 0.2983260869565243,
 'Coverage[0.2]': 0.41576086956521907,
 'Coverage[0.3]': 0.5034782608695669,
 'Coverage[0.4]': 0.5788840579710149,
 'Coverage[0.5]': 0.6447246376811628,
 'Coverage[0.6]': 0.700079710144935,
 'Coverage[0.7]': 0.7594565217391374,
 'Coverage[0.8]': 0.8192173913043503,
 'Coverage[0.9]': 0.8823985507246397,
 'MAE_Coverage': 0.1263921095008079,
 'MASE': 3.394805832177617,
 'MSE': 3256187.6730807004,
 'MSIS': 52.95574707055701,
 'ND': 0.14067211186883702,
 'NRMSE': 0.28931406315189456,
 'QuantileLoss[0.1]': 61741598.68876076,
 'QuantileLoss[0.2]': 89675151.18205567,
 'QuantileLoss[0.3]': 107140824.1709732,
 'QuantileLoss[0.4]': 117269982.81333007,
 'QuantileLoss[0.5]': 121079958.51117706,
 'QuantileLoss[0.6]': 118407274.55362853,
 'QuantileLoss[0.7]': 109752818.08506928,
 'QuantileLoss[0.8]': 93584218.54154052,
 'QuantileLoss[0.9]': 66568238.93393707,
 'RMSE': 1804.4909733996178,
 'abs_error': 121079958.42617798,
 'abs_target_mean': 6237.135359895143,
 'a

,dataset,estimator,MASE,sMAPE,MSIS,wQuantileLoss[0.5],wQuantileLoss[0.9]
0,m4_quarterly,DeepAREstimator,1.170435,0.100674,11.356949,0.093112,0.051371
1,m4_yearly,DeepAREstimator,3.394806,0.143477,52.955747,0.140672,0.077340
